In [1]:
#initial imports 
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Optional
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load Files as DataFrames
BASE_DIR = Path.cwd().resolve().parents[1]
data_file = BASE_DIR / "data" / "feature" / "kickstarter_featured.csv"

filepath = Path(data_file)

df = pd.read_csv(filepath, encoding='latin-1', low_memory=False)

In [3]:
df.head()

,id,main_category,deadline,launched,backers,country,usd_pledged_real,usd_goal_real,duration_days,target,...,launched_year,launched_month,duration_bins,backers_per_pledged,backer_pledged_bins,pledged_per_category,goal_per_category,category_goal_percentile,launch_season,deadline_season
0,1000003930,Film & Video,2017-11-01,2017-09-02 04:43:57,15,US,2421.0,30000.0,59,0,...,2017,9,6 weeks,0.006196,Very Low,7676.247109,58616.915835,Very High,Fall,Fall
1,1000004038,Film & Video,2013-02-26,2013-01-12 00:20:50,3,US,220.0,45000.0,44,0,...,2013,1,4 weeks,0.013636,Low,7676.247109,58616.915835,Very High,Winter,Winter
2,1000007540,Music,2012-04-16,2012-03-17 03:24:11,1,US,1.0,5000.0,29,0,...,2012,3,2 weeks,1.000000,Very High,4697.431965,11558.623284,Medium,Spring,Spring
3,1000014025,Food,2016-04-01,2016-02-26 13:38:27,224,US,52375.0,50000.0,34,1,...,2016,2,4 weeks,0.004277,Very Low,6505.672844,30502.224195,Very High,Winter,Spring
4,1000023410,Food,2014-12-21,2014-12-01 18:30:44,16,US,1205.0,1000.0,19,1,...,2014,12,2 weeks,0.013278,Low,6505.672844,30502.224195,Very Low,Winter,Winter


In [4]:
df.columns

Index(['id', 'main_category', 'deadline', 'launched', 'backers', 'country',
       'usd_pledged_real', 'usd_goal_real', 'duration_days', 'target',
       'usd_goal_bins', 'usd_pledged_bins', 'main_category_grouped',
       'continent', 'deadline_year', 'deadline_month', 'launched_year',
       'launched_month', 'duration_bins', 'backers_per_pledged',
       'backer_pledged_bins', 'pledged_per_category', 'goal_per_category',
       'category_goal_percentile', 'launch_season', 'deadline_season'],
      dtype='object')